# Análisis predictivo de la disponibilidad de bicicletas en estaciones BiciMad
Pablo Ariño & Álvaro Laguna

## Importación de librerías y cofiguración

In [1]:
import os

import dotenv

from src.data import (
    find_json_files_in_directory,
    load_json_files,
    load_json_objects,
    build_stations_dataframe,
    get_k_closest_stations,
    build_dock_bikes_timeseries_dataframe
)
from src.visualization import (
    plot_stations_in_map)

dotenv.load_dotenv()

True

In [2]:
DATA_PATH = os.getenv("DATA_PATH", "data")
DATA_RAW_PATH = DATA_PATH + "/raw/"
DATA_FILES = sorted(find_json_files_in_directory(DATA_RAW_PATH))
print(f"Data files: {DATA_FILES}")
TARGET_STATION_ID = int(os.getenv("TARGET_STATION_ID", "90"))
NUM_NEIGHBOR_STATIONS = int(os.getenv("NUM_NEIGHBOR_STATIONS", "5"))

Data files: ['202210.json', '202211.json', '202212.json']


## Load data

In [3]:
raw_data = load_json_files(DATA_RAW_PATH, DATA_FILES)

In [4]:
len(raw_data)

2181

## Process data

In [5]:
stations = build_stations_dataframe(raw_data)
stations

,name,number,address,latitude,longitude,total_bases
id,,,,,,
1,Puerta del Sol A,1a,Puerta del Sol nº 1,40.4172137,-3.7018341,30
2,Puerta del Sol B,1b,Puerta del Sol nº 1,40.41731271011562,-3.701602938060457,30
3,Miguel Moya,2,Calle Miguel Moya nº 1,40.4205886,-3.7058415,24
4,Plaza Conde Suchil,3,Plaza del Conde del Valle de Súchil nº 3,40.4302937,-3.7069171,18
5,Malasaña,4,Calle Manuela Malasaña nº 5,40.4285524,-3.7025875,24
...,...,...,...,...,...,...
265,INEF,257,Avenida Juan de Herrera frente a la calle Paul...,40.43896,-3.72997,24
266,Ciudad Universitaria 1,258,Avenida de la Complutense (Metro Ciudad Univer...,40.44375,-3.72699,24
267,Ciudad Universitaria 2,259,Avenida de la Complutense (Metro Ciudad Univer...,40.44342,-3.72693,24


In [6]:
top_5_closest_stations = get_k_closest_stations(
    TARGET_STATION_ID, stations, k=NUM_NEIGHBOR_STATIONS,
)

In [7]:
top_5_closest_stations

,name,number,address,latitude,longitude,total_bases,distance
id,,,,,,,
102,Alcalá,97,Avenida de Menéndez Pelayo nº 3,40.4222969,-3.6805189,27,0.098442
101,Castelló,96,Calle Alcalá nº 111,40.4220640,-3.6821793,27,0.194586
107,Velázquez,102,Calle Alcalá nº 95,40.4211802,-3.6840229,24,0.342588
65,Narváez,61,Calle O'Donnell nº 28,40.4213983,-3.6752045,24,0.407816
103,Plaza de Felipe II,98,Plaza de Felipe II,40.4239447,-3.6758383,24,0.445928


In [8]:
top_5_closest_stations_ids = top_5_closest_stations.index.tolist()
plot_stations_in_map(
    stations,
    target_station_id=TARGET_STATION_ID,
    other_stations_ids=top_5_closest_stations_ids,
)

In [9]:
station_names = [TARGET_STATION_ID] + top_5_closest_stations_ids
dock_bikes_timeseries = build_dock_bikes_timeseries_dataframe(
    raw_data, station_names
)

In [10]:
dock_bikes_timeseries

,65,90,101,102,103,107
2022-10-01 00:05:05.771548,6,10,6,9,3,1
2022-10-01 01:05:08.526286,5,10,13,6,2,1
2022-10-01 02:05:09.305373,6,10,10,4,2,3
2022-10-01 03:05:10.046779,6,9,10,3,2,1
2022-10-01 04:05:11.827629,6,8,10,4,4,1
...,...,...,...,...,...,...
2022-12-31 19:57:50.302819,11,8,12,12,2,5
2022-12-31 20:57:50.382304,9,4,11,11,2,6
2022-12-31 21:57:52.761035,11,3,13,13,1,6
2022-12-31 22:57:53.608384,9,3,13,13,1,6
